In [2]:
# first, test generative model on one dimensional graphs
import sys
sys.path.append('./HCM') # outdated due to abstraction learning implementation
import pandas as pd
import seaborn as sns
import numpy as np
from Learning import *
from CG1 import *

In [4]:
import pandas as pd
df = pd.read_csv('/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/data.csv')

dfm = {}  # model dataframe
dfm['blockcollect'] = []
dfm['ID'] = []
dfm['condition'] = []
dfm['correctcollect'] = []
dfm['p'] = []
dfm['trialcollect'] = []
dfm['recall_likelihood'] = [] # the likelihood of subjects recalling the exact piece of sequence, given the representation learned so far

column_names = ['ID', 'condition', 'recallcollect', 'instructioncollect','trialcollect', 'correctcollect']
dfs = pd.DataFrame(columns=column_names)  # Note that there are now row data inserted.


seql = 12
len_train = 40
len_test = 8

def convert_sequence(seq,keyassignment):
    keyassignment = list(keyassignment)
    ka = [keyassignment[2],keyassignment[7],keyassignment[12],keyassignment[17],keyassignment[22],keyassignment[27]]
    proj_seq = [ka.index(item)+ 1 for item in seq]
    return proj_seq

def convert_sequence_backward_to_key(seq, keyassignment):
    keyassignment = list(keyassignment)
    inst_seq = []
    ka = [keyassignment[2], keyassignment[7], keyassignment[12], keyassignment[17], keyassignment[22],
          keyassignment[27]]
    for i in range(0, 12):
        inst_seq.append(ka[int(seq[i,0,0])-1])
    return inst_seq


def calculate_prob(chunk_record, cg):
    p = 1
    for key in list(chunk_record.keys()):# key is the encoding time
        p = p*cg.chunks[chunk_record[key][0][0]].count/np.sum([item.count for item in cg.chunks])
    return p
    

for sub in np.unique(list(df['ID'])):
    # initialize chunking part with specified parameters
    cg = CG1(DT=0.1, theta=0.996)
    for trial in range(1, len_train + 3*len_test+ 1):
        ins_seq = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'instructioncollect']
        subj_recall =  df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'recallcollect']
        keyassignment = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'keyassignment'])[0]
        condition = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'condition'])[0]
        block = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'blockcollect'])[0]
        proj_seq = convert_sequence(list(ins_seq),keyassignment)
        #print(proj_seq)
        proj_seq = np.array(proj_seq).reshape([-1, 1, 1])
    
        cg, chunkrecord = hcm_learning(proj_seq, cg,chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recalled_seq,ps = recall(cg, firstitem=proj_seq[0,0,0])   
        keypress_recalled_seq = convert_sequence_backward_to_key(recalled_seq,keyassignment)    
        
        p_seq = np.prod(ps)# evaluate the probability of a sequence
        proj_recalled_seq = convert_sequence(list(subj_recall),keyassignment)
        proj_recalled_seq = np.array(proj_recalled_seq).reshape([-1, 1, 1])
        cg, chunkrecord = hcm_learning(proj_recalled_seq, cg, learn = False, chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recall_likelihood = calculate_prob(chunkrecord, cg)# evaluate the probability of a sequence
        print(recall_likelihood)
        
        dfm['blockcollect'].append(block)
        dfm['ID'].append(sub)
        dfm['condition'].append(condition)
        dfm['correctcollect'].append(acc_eval1d(recalled_seq, proj_seq))
        dfm['p'].append(p_seq)
        dfm['trialcollect'].append(trial)
        dfm['recall_likelihood'].append(recall_likelihood)
        
        ins_list = ins_seq.tolist()
        print(keypress_recalled_seq, ins_list)
        for i in range(0, 12):
            if keypress_recalled_seq[i] == ins_list[i]:
                correctcollect = 1
            else: correctcollect = 0
            dfs = dfs.append({'ID': sub,
                            'condition': condition,
                            'keyassignment': keyassignment,
                            'recallcollect': keypress_recalled_seq[i],
                            'trialcollect': trial,
                            'instructioncollect': ins_list[i],
                            'correctcollect': correctcollect}, ignore_index=True)

dfm = pd.DataFrame.from_dict(dfm)
csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/simulation_data_model_transition_recall_associative_learning.csv'
dfm.to_csv(csv_save_directory, index=False, header=True)


dfs_csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/simulation_data_model_transition_recall_associative_learning_individualkey.csv'
dfs.to_csv(dfs_csv_save_directory, index=False, header=True)



4.375375182304919e-10
accuracy is:  0.75
['J', 'D', 'S', 'L', 'L', 'K', 'J', 'D', 'S', 'L', 'L', 'L'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
6.078768801064287e-10
accuracy is:  0.4166666666666667
['J', 'D', 'S', 'L', 'L', 'L', 'L', 'L', 'J', 'D', 'S', 'F'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
4.5863180177297153e-10
accuracy is:  0.3333333333333333
['J', 'D', 'S', 'L', 'J', 'D', 'S', 'L', 'L', 'K', 'J', 'D'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
1.05176924378038e-09
accuracy is:  0.6666666666666666
['J', 'D', 'S', 'L', 'L', 'L', 'J', 'D', 'S', 'L', 'L', 'K'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
1.633367555114977e-09
accuracy is:  0.25
['J', 'D', 'S', 'S', 'S', 'L', 'L', 'L', 'L', 'K', 'J', 'D'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
1.7371490104594799e-09
accuracy is:  0.75
['J', 'D', 'S', 'L', 'L', 'K', 'J', 'D', 'S', 'L', 'L', 'L'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S',

1.0645679783906129e-09
accuracy is:  0.3333333333333333
['S', 'L', 'L', 'L', 'K', 'S', 'S', 'D', 'L', 'L', 'J', 'D'] ['S', 'D', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'K', 'S', 'J']
8.917392396198154e-10
accuracy is:  0.4166666666666667
['S', 'L', 'L', 'K', 'J', 'D', 'L', 'L', 'F', 'L', 'F', 'L'] ['S', 'F', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'S', 'J']
4.266799864512161e-09
accuracy is:  0.25
['S', 'D', 'S', 'L', 'J', 'K', 'J', 'S', 'D', 'S', 'L', 'D'] ['S', 'D', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'K', 'S', 'J']
1.7887278015587956e-09
accuracy is:  0.3333333333333333
['S', 'L', 'L', 'J', 'S', 'J', 'D', 'S', 'L', 'J', 'J', 'F'] ['S', 'K', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'F', 'S', 'J']
3.101007291263897e-09
accuracy is:  0.16666666666666666
['S', 'L', 'J', 'S', 'D', 'S', 'L', 'K', 'J', 'J', 'S', 'S'] ['S', 'D', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'K', 'S', 'J']
4.743853482310495e-09
accuracy is:  0.16666666666666666
['S', 'L', 'J', 'D', 'S', 'S', 'J', 'S', 'L', 'J', 'J', 'D'] ['S', 'F'

2.3599596884321435e-09
accuracy is:  0.25
['K', 'D', 'S', 'K', 'F', 'S', 'F', 'K', 'F', 'K', 'F', 'S'] ['K', 'D', 'S', 'L', 'S', 'F', 'K', 'L', 'L', 'J', 'S', 'K']
9.915032740561976e-10
accuracy is:  0.5
['K', 'F', 'S', 'L', 'L', 'S', 'L', 'S', 'K', 'F', 'S', 'K'] ['K', 'J', 'S', 'L', 'S', 'D', 'K', 'L', 'L', 'F', 'S', 'K']
3.289061333554052e-10
accuracy is:  0.3333333333333333
['K', 'D', 'K', 'L', 'D', 'S', 'D', 'S', 'L', 'F', 'S', 'L'] ['K', 'F', 'S', 'L', 'S', 'J', 'K', 'L', 'L', 'D', 'S', 'K']
1.126479810996255e-09
accuracy is:  0.16666666666666666
['K', 'F', 'S', 'K', 'L', 'L', 'S', 'K', 'J', 'K', 'F', 'S'] ['K', 'J', 'S', 'L', 'S', 'F', 'K', 'L', 'L', 'D', 'S', 'K']
9.968578403466748e-10
accuracy is:  0.25
['K', 'L', 'L', 'L', 'D', 'K', 'L', 'S', 'L', 'L', 'J', 'S'] ['K', 'F', 'S', 'L', 'S', 'D', 'K', 'L', 'L', 'J', 'S', 'K']
2.0738345537720344e-09
accuracy is:  0.08333333333333333
['K', 'L', 'L', 'S', 'J', 'S', 'L', 'J', 'S', 'L', 'J', 'S'] ['K', 'F', 'S', 'L', 'S', 'J', 'K', 'L

5.864957775644545e-08
accuracy is:  0.3333333333333333
['K', 'S', 'K', 'F', 'J', 'F', 'J', 'F', 'J', 'F', 'S', 'J'] ['K', 'L', 'J', 'F', 'J', 'S', 'F', 'K', 'F', 'D', 'K', 'J']
8.333549016441002e-08
accuracy is:  0.3333333333333333
['K', 'J', 'L', 'K', 'S', 'J', 'F', 'K', 'S', 'K', 'F', 'J'] ['K', 'S', 'J', 'F', 'J', 'L', 'F', 'K', 'F', 'D', 'K', 'J']
1.485880587797929e-07
accuracy is:  0.08333333333333333
['K', 'F', 'K', 'L', 'K', 'F', 'K', 'S', 'J', 'S', 'J', 'F'] ['K', 'D', 'J', 'F', 'J', 'S', 'F', 'K', 'F', 'L', 'K', 'J']
5.127018098918383e-09
accuracy is:  0.5833333333333334
['K', 'S', 'J', 'F', 'S', 'J', 'F', 'K', 'J', 'F', 'K', 'F'] ['K', 'S', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'L', 'K', 'J']
3.1315794063855097e-10
accuracy is:  0.25
['K', 'F', 'K', 'F', 'K', 'S', 'J', 'L', 'F', 'J', 'F', 'D'] ['K', 'L', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'S', 'K', 'J']
1.1188002764159533e-10
accuracy is:  0.16666666666666666
['K', 'F', 'K', 'F', 'S', 'J', 'L', 'F', 'S', 'J', 'L', 'K'] ['K', 'D'

8.802376111352732e-10
accuracy is:  0.08333333333333333
['K', 'D', 'D', 'L', 'S', 'D', 'J', 'K', 'D', 'D', 'F', 'K'] ['K', 'J', 'S', 'D', 'D', 'F', 'K', 'S', 'S', 'L', 'K', 'D']
1.0142117516674504e-09
accuracy is:  0.3333333333333333
['K', 'D', 'L', 'S', 'D', 'D', 'D', 'D', 'D', 'F', 'K', 'S'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
8.762047827180044e-10
accuracy is:  0.4166666666666667
['K', 'J', 'S', 'S', 'D', 'D', 'D', 'D', 'D', 'F', 'K', 'S'] ['K', 'J', 'S', 'D', 'D', 'F', 'K', 'S', 'S', 'L', 'K', 'D']
4.792758530225196e-10
accuracy is:  0.16666666666666666
['K', 'D', 'D', 'F', 'K', 'J', 'K', 'L', 'K', 'S', 'D', 'J'] ['K', 'J', 'S', 'D', 'D', 'L', 'K', 'S', 'S', 'F', 'K', 'D']
1.233453688840274e-09
accuracy is:  0.25
['K', 'D', 'J', 'S', 'D', 'D', 'D', 'J', 'K', 'L', 'K', 'J'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
8.752567837044192e-10
accuracy is:  0.08333333333333333
['K', 'J', 'K', 'J', 'S', 'D', 'D', 'F', 'K', 'D', 'D', 'L'] ['K', 'F'

...

2.156529273076633e-09
accuracy is:  0.3333333333333333
['F', 'D', 'K', 'F', 'S', 'F', 'S', 'J', 'S', 'F', 'D', 'S'] ['F', 'L', 'K', 'S', 'S', 'D', 'K', 'F', 'K', 'J', 'F', 'S']
2.153903592397616e-09
accuracy is:  0.4166666666666667
['F', 'L', 'K', 'S', 'K', 'L', 'K', 'D', 'K', 'F', 'K', 'F'] ['F', 'D', 'K', 'S', 'S', 'J', 'K', 'F', 'K', 'L', 'F', 'S']
2.1594961824235455e-09
accuracy is:  0.25
['F', 'S', 'K', 'F', 'S', 'J', 'S', 'K', 'F', 'S', 'S', 'F'] ['F', 'D', 'K', 'S', 'S', 'L', 'K', 'F', 'K', 'J', 'F', 'S']
2.1643324806889864e-09
accuracy is:  0.4166666666666667
['F', 'S', 'D', 'S', 'F', 'J', 'S', 'L', 'K', 'S', 'F', 'S'] ['F', 'L', 'K', 'S', 'S', 'D', 'K', 'F', 'K', 'J', 'F', 'S']
8.94158353034701e-10
accuracy is:  0.16666666666666666
['F', 'D', 'K', 'J', 'F', 'S', 'F', 'K', 'S', 'F', 'L', 'F'] ['F', 'L', 'K', 'S', 'S', 'J', 'K', 'F', 'K', 'D', 'F', 'S']
3.2192578568938435e-10
accuracy is:  0.4166666666666667
['F', 'D', 'K', 'S', 'D', 'K', 'F', 'S', 'K', 'S', 'F', 'D'] ['F', 'J',

4.881335743920436e-10
accuracy is:  0.08333333333333333
['L', 'F', 'L', 'D', 'L', 'K', 'S', 'F', 'S', 'K', 'F', 'S'] ['L', 'D', 'S', 'K', 'S', 'J', 'L', 'K', 'K', 'F', 'S', 'L']
4.97469790736912e-10
accuracy is:  0.25
['L', 'J', 'L', 'J', 'L', 'J', 'L', 'D', 'S', 'K', 'S', 'K'] ['L', 'F', 'S', 'K', 'S', 'D', 'L', 'K', 'K', 'J', 'S', 'L']
2.8499561023678007e-10
accuracy is:  0.3333333333333333
['L', 'D', 'S', 'K', 'K', 'S', 'K', 'K', 'S', 'K', 'J', 'S'] ['L', 'J', 'S', 'K', 'S', 'F', 'L', 'K', 'K', 'D', 'S', 'L']
2.9769640964894207e-10
accuracy is:  0.3333333333333333
['L', 'K', 'S', 'K', 'S', 'K', 'F', 'S', 'L', 'K', 'K', 'K'] ['L', 'J', 'S', 'K', 'S', 'D', 'L', 'K', 'K', 'F', 'S', 'L']
1.7487919547474748e-09
accuracy is:  0.5
['L', 'J', 'L', 'D', 'S', 'J', 'L', 'J', 'L', 'D', 'S', 'K'] ['L', 'F', 'S', 'K', 'S', 'J', 'L', 'K', 'K', 'D', 'S', 'L']
6.727915490021261e-10
accuracy is:  0.16666666666666666
['L', 'K', 'K', 'S', 'J', 'L', 'D', 'S', 'K', 'K', 'J', 'S'] ['L', 'D', 'S', 'K', 'S'

5.0122479080681e-10
accuracy is:  0.4166666666666667
['D', 'K', 'S', 'F', 'S', 'K', 'S', 'K', 'D', 'L', 'S', 'K'] ['D', 'J', 'S', 'K', 'S', 'F', 'K', 'D', 'K', 'L', 'S', 'D']
1.0233916214713941e-09
accuracy is:  0.25
['D', 'J', 'S', 'L', 'S', 'K', 'S', 'F', 'S', 'K', 'L', 'K'] ['D', 'L', 'S', 'K', 'S', 'F', 'K', 'D', 'K', 'J', 'S', 'D']
4.3285861611703667e-10
accuracy is:  0.3333333333333333
['D', 'J', 'S', 'J', 'S', 'K', 'S', 'D', 'L', 'S', 'K', 'L'] ['D', 'F', 'S', 'K', 'S', 'L', 'K', 'D', 'K', 'J', 'S', 'D']
4.374128446037089e-10
accuracy is:  0.3333333333333333
['D', 'L', 'S', 'K', 'F', 'K', 'S', 'K', 'S', 'L', 'K', 'S'] ['D', 'L', 'S', 'K', 'S', 'F', 'K', 'D', 'K', 'J', 'S', 'D']
3.941968892188891e-10
accuracy is:  0.25
['D', 'L', 'S', 'J', 'K', 'D', 'L', 'S', 'K', 'F', 'K', 'S'] ['D', 'J', 'S', 'K', 'S', 'F', 'K', 'D', 'K', 'L', 'S', 'D']
1.089017804182007e-09
accuracy is:  0.4166666666666667
['D', 'K', 'D', 'K', 'S', 'L', 'S', 'L', 'K', 'S', 'K', 'F'] ['D', 'J', 'S', 'K', 'S', '

3.9142336208494457e-10
accuracy is:  0.25
['K', 'D', 'J', 'K', 'S', 'D', 'J', 'S', 'K', 'D', 'K', 'S'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
6.029424524829598e-10
accuracy is:  0.5
['K', 'D', 'S', 'J', 'S', 'J', 'S', 'J', 'S', 'J', 'S', 'J'] ['K', 'D', 'S', 'J', 'S', 'F', 'J', 'K', 'J', 'L', 'S', 'K']
7.903515068047724e-10
accuracy is:  0.3333333333333333
['K', 'D', 'S', 'F', 'J', 'S', 'J', 'S', 'J', 'S', 'J', 'S'] ['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'L', 'S', 'K']
2.0676530072024888e-09
accuracy is:  0.3333333333333333
['K', 'D', 'S', 'J', 'K', 'D', 'S', 'D', 'S', 'K', 'D', 'S'] ['K', 'D', 'S', 'J', 'S', 'F', 'J', 'K', 'J', 'L', 'S', 'K']
2.0917583214404384e-09
accuracy is:  0.5833333333333334
['K', 'D', 'S', 'J', 'S', 'J', 'L', 'S', 'J', 'L', 'S', 'J'] ['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'L', 'S', 'K']
6.095696811604354e-09
accuracy is:  0.5
['K', 'J', 'S', 'D', 'S', 'J', 'L', 'S', 'K', 'D', 'S', 'K'] ['K', 'L', 'S', 'J', 'S', 'F', 'J', 'K',

2.17500355140316e-09
accuracy is:  0.4166666666666667
['J', 'D', 'J', 'K', 'L', 'S', 'J', 'S', 'L', 'S', 'S', 'K'] ['J', 'D', 'K', 'S', 'K', 'F', 'J', 'S', 'S', 'L', 'J', 'K']
2.1785965847917013e-09
accuracy is:  0.3333333333333333
['J', 'K', 'D', 'J', 'D', 'J', 'D', 'S', 'K', 'S', 'J', 'K'] ['J', 'D', 'K', 'S', 'K', 'L', 'J', 'S', 'S', 'F', 'J', 'K']
8.808401894958697e-10
accuracy is:  0.25
['J', 'K', 'F', 'J', 'K', 'D', 'S', 'F', 'S', 'F', 'S', 'F'] ['J', 'F', 'K', 'S', 'K', 'L', 'J', 'S', 'S', 'D', 'J', 'K']
2.1754647688529977e-09
accuracy is:  0.08333333333333333
['J', 'S', 'J', 'F', 'J', 'S', 'K', 'J', 'K', 'J', 'F', 'J'] ['J', 'L', 'K', 'S', 'K', 'D', 'J', 'S', 'S', 'F', 'J', 'K']
7.454187489172936e-10
accuracy is:  0.16666666666666666
['J', 'S', 'J', 'S', 'L', 'J', 'S', 'L', 'J', 'S', 'S', 'J'] ['J', 'F', 'K', 'S', 'K', 'D', 'J', 'S', 'S', 'L', 'J', 'K']
2.1817439202698404e-09
accuracy is:  0.25
['J', 'S', 'S', 'L', 'J', 'F', 'J', 'K', 'D', 'S', 'D', 'J'] ['J', 'L', 'K', 'S', 'K